In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from IPython.display import Image
import pandas as pd

import re

In [3]:
# 웹드라이브로 크롬브라우즈 띄운다.
driver_path = "driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)

In [4]:
url_page = 'https://www.starbucks.co.kr/index.do'
driver.get(url_page)

# 서울지역 스타벅스 매장정보 크롤링

### 매장 찾기 사이트 이동

In [5]:
url_page = 'https://www.starbucks.co.kr/store/store_map.do'
driver.get(url_page)

### 지역검색 클릭

In [7]:
driver.find_element_by_xpath(
    '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]/h3/a').click()

### 서울 클릭

In [8]:
driver.find_element_by_xpath(
    '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a').click()

### 전체 클릭

In [9]:
driver.find_element_by_xpath(
    '//*[@id="mCSB_2_container"]/ul/li[1]/a').click()

In [10]:
html = driver.page_source
soup = BeautifulSoup(html, "lxml")

### 데이터 확인

In [11]:
review_cnt = soup.find("ul","quickSearchResultBoxSidoGugun")
review_cnt

<ul class="quickSearchResultBoxSidoGugun"><li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong>역삼아레나빌딩  <img alt="" class="setStoreFavBtn mCS_img_loaded" data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N" src="//image.istarbucks.co.kr/common/img/store/icon_fav_off.png"/></strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li><li class="quickResultLstCon" data-code="3672" data-hlytag="null" data-index="1" data-lat="37.510178" data-long="127.022223" data-name="논현역사거리" data-storecd="1434" style="background:#fff"> <strong>논현역사거리  <img alt="" class="setStoreFavBtn mCS_img_loaded" data-my_siren_order_store_yn="N" data-name="논현역사거리" data-store="1434" data-yn="N" src="//image.istarbucks.co.kr/common/img/store/icon_fav_off.png"/></strong> <p class="result_det

In [12]:
re1 = review_cnt.find_all('li')
re1[0].get_text()

' 역삼아레나빌딩   서울특별시 강남구 언주로 425 (역삼동)1522-3232 리저브 매장 2번'

## 스타벅스 매장 이름

In [14]:
starbucks_ul = soup.find("ul","quickSearchResultBoxSidoGugun")
starbucks_li = review_cnt.find_all('li')
starbucks_li[0]['data-name']

'역삼아레나빌딩'

## 스타벅스 매장 주소

In [15]:
starbucks_ul = soup.find("ul","quickSearchResultBoxSidoGugun")
starbucks_li = review_cnt.find_all('li')
starbucks_li[0].find('p','result_details').get_text()[:-9]


'서울특별시 강남구 언주로 425 (역삼동)'

## 매장 소재 구

In [16]:
starbucks_ul = soup.find("ul","quickSearchResultBoxSidoGugun")
starbucks_li = review_cnt.find_all('li')
starbucks_li[0].find('p','result_details').get_text().split()[1]

'강남구'

## 스타벅스 전화번호

In [17]:
starbucks_ul = soup.find("ul","quickSearchResultBoxSidoGugun")
starbucks_li = review_cnt.find_all('li')
starbucks_li[0].find('p','result_details').get_text()[-9:]

'1522-3232'

In [18]:
sbucksName = []
sbucksGu = []
sbucksAddr = []
sbucksTel = []

starbucks_ul = soup.find("ul","quickSearchResultBoxSidoGugun")
starbucks_li = review_cnt.find_all('li')

for i in range(0,len(review_cnt.find_all('li'))):  
    sbucksName.append(starbucks_li[i]['data-name'])
    sbucksAddr.append(starbucks_li[i].find('p','result_details').get_text()[:-9])
    sbucksGu.append(starbucks_li[i].find('p','result_details').get_text().split()[1])
    sbucksTel.append(starbucks_li[i].find('p','result_details').get_text()[-9:])

starbucks = pd.DataFrame({'스타벅스매장':sbucksName, 
                          '구'          :sbucksGu, 
                          '주소'        :sbucksAddr, 
                          '전화번호'    :sbucksTel, 
                         })

In [19]:
starbucks.head(10)

,스타벅스매장,구,주소,전화번호
0,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,강남구,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,강남구,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,강남구,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,강남구,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
5,봉은사역,강남구,서울특별시 강남구 봉은사로 619 (삼성동),1522-3232
6,압구정윤성빌딩,강남구,서울특별시 강남구 논현로 834 (신사동),1522-3232
7,코엑스별마당,강남구,서울특별시 강남구 영동대로 513 (삼성동),1522-3232
8,삼성역섬유센터R,강남구,서울특별시 강남구 테헤란로 518 (대치동),1522-3232
9,압구정R,강남구,서울특별시 강남구 언주로 861 (신사동),1522-3232


# 서울인구 정보를 가져온 후 스타벅스지표를 나타낸다.

## 인구데이터 불러오기

In [20]:
df_pop = pd.read_excel('data/Report_seoul_population_2019_2Q.xls')
df=df_pop[['자치구','세대','인구','세대당인구','65세이상고령자']][-25:]
df['인구'].astype(int)
df['65세이상고령자'].astype(int)
df.head()

,자치구,세대,인구,세대당인구,65세이상고령자
3,종로구,73828,162413,2.06,27155
4,중구,62156,136276,2.03,22749
5,용산구,109692,245069,2.09,38259
6,성동구,136872,312991,2.23,43370
7,광진구,163914,369470,2.16,46773


## 고령비율계산

In [21]:
df['고령비율']= df['인구']/df['65세이상고령자']

In [22]:
df.set_index('자치구', inplace=True)        

In [23]:
df.head()

,세대,인구,세대당인구,65세이상고령자,고령비율
자치구,,,,,
종로구,73828,162413,2.06,27155,5.98096
중구,62156,136276,2.03,22749,5.99042
용산구,109692,245069,2.09,38259,6.40553
성동구,136872,312991,2.23,43370,7.21676
광진구,163914,369470,2.16,46773,7.89922


## 스벅비율 계산

In [24]:
df_sb=starbucks.groupby('구').count()['스타벅스매장']

In [25]:
df = pd.merge(df,df_sb,left_index=True,right_index=True)

In [26]:
df.rename(columns = {'스타벅스매장' : '스타벅스'}, inplace = True)
df['스벅비율'] = 1000000*df['스타벅스']/df['인구']
df.sort_values(by='스타벅스', ascending=False).head()

,세대,인구,세대당인구,65세이상고령자,고령비율,스타벅스,스벅비율
강남구,230752,547602,2.35,69011,7.935,84,153.396
중구,62156,136276,2.03,22749,5.99042,54,396.255
서초구,172907,435631,2.49,55853,7.7996,47,107.889
종로구,73828,162413,2.06,27155,5.98096,38,233.971
영등포구,173701,404229,2.12,56891,7.10532,34,84.1107


## 시각화를 위한 Folium 불러오기

In [27]:
import folium

import pandas as pd
import json
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

In [28]:
geo_path = 'data/skorea_municipalities_geo_simple.json'
geo_str  = json.load(open(geo_path, encoding='utf-8'))

# 스벅비율 및 스벅매장수를 서울시 지도에 시각화

In [29]:
map = folium.Map(location=[37.5502, 126.982], tiles='Stamen Toner', zoom_start=11)

map.choropleth(geo_data=geo_str,
               data=df,
               columns=[df.index, '스타벅스'],
               fill_color='PuRd', 
               key_on='feature.id')
map

# 인구대비 스벅비율

In [30]:
map = folium.Map(location=[37.5502, 126.982], tiles='Stamen Toner', zoom_start=11)

map.choropleth(geo_data=geo_str,
               data=df,
               columns=[df.index, '스벅비율'],
               fill_color='YlGnBu', 
               key_on='feature.id')
map

# 스벅비율이 반영된 파일을 저장

In [31]:
df.to_csv('data/seoul_population_sbucks-idx.csv', sep=',', encoding='euc-kr')

In [32]:
df

,세대,인구,세대당인구,65세이상고령자,고령비율,스타벅스,스벅비율
종로구,73828,162413,2.06,27155,5.98096,38,233.971
중구,62156,136276,2.03,22749,5.99042,54,396.255
용산구,109692,245069,2.09,38259,6.40553,20,81.6097
성동구,136872,312991,2.23,43370,7.21676,11,35.1448
광진구,163914,369470,2.16,46773,7.89922,16,43.3053
동대문구,162851,363101,2.13,58078,6.25195,8,22.0324
중랑구,181383,405639,2.21,63670,6.37096,8,19.722
성북구,190253,452292,2.31,69341,6.52272,13,28.7425
강북구,143650,319377,2.2,59189,5.39588,5,15.6555
도봉구,138460,339001,2.43,57367,5.90934,2,5.89969


# 인구가 많은 순으로 정렬

In [35]:
df.sort_values(by=["인구"],ascending=False)

,세대,인구,세대당인구,65세이상고령자,고령비율,스타벅스,스벅비율
송파구,278223,686181,2.44,84740,8.09749,32,46.6349
강서구,261383,602230,2.28,81965,7.3474,16,26.5679
강남구,230752,547602,2.35,69011,7.935,84,153.396
노원구,217462,542603,2.48,79046,6.8644,12,22.1156
관악구,266692,519864,1.88,73693,7.05446,11,21.1594
은평구,207123,488285,2.34,79282,6.15884,8,16.3839
양천구,177045,465095,2.6,59731,7.78649,15,32.2515
성북구,190253,452292,2.31,69341,6.52272,13,28.7425
구로구,174505,439328,2.32,64028,6.8615,11,25.0382
서초구,172907,435631,2.49,55853,7.7996,47,107.889


# 스벅이 많은 순으로 정렬

In [36]:
df.sort_values(by=["스타벅스"],ascending=False)

,세대,인구,세대당인구,65세이상고령자,고령비율,스타벅스,스벅비율
강남구,230752,547602,2.35,69011,7.935,84,153.396
중구,62156,136276,2.03,22749,5.99042,54,396.255
서초구,172907,435631,2.49,55853,7.7996,47,107.889
종로구,73828,162413,2.06,27155,5.98096,38,233.971
영등포구,173701,404229,2.12,56891,7.10532,34,84.1107
마포구,173845,385985,2.15,51613,7.47845,33,85.4956
송파구,278223,686181,2.44,84740,8.09749,32,46.6349
용산구,109692,245069,2.09,38259,6.40553,20,81.6097
서대문구,140007,324304,2.22,51367,6.31347,19,58.587
강서구,261383,602230,2.28,81965,7.3474,16,26.5679


# 인구대비 스타벅스가 많은 순으로 정렬

In [37]:
df.sort_values(by=["스벅비율"],ascending=False)

,세대,인구,세대당인구,65세이상고령자,고령비율,스타벅스,스벅비율
중구,62156,136276,2.03,22749,5.99042,54,396.255
종로구,73828,162413,2.06,27155,5.98096,38,233.971
강남구,230752,547602,2.35,69011,7.935,84,153.396
서초구,172907,435631,2.49,55853,7.7996,47,107.889
마포구,173845,385985,2.15,51613,7.47845,33,85.4956
영등포구,173701,404229,2.12,56891,7.10532,34,84.1107
용산구,109692,245069,2.09,38259,6.40553,20,81.6097
서대문구,140007,324304,2.22,51367,6.31347,19,58.587
송파구,278223,686181,2.44,84740,8.09749,32,46.6349
광진구,163914,369470,2.16,46773,7.89922,16,43.3053
